# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Load Libraries

In [1]:
import numpy as np
import evaluate
import torch
from datasets import load_dataset
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification, DistilBertModel
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from peft import LoraConfig, PeftModel, get_peft_model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AutoModelForSequenceClassification
from peft import AutoPeftModelForSequenceClassification

### Load Dataset

In [2]:
dataset = load_dataset("sms_spam")

In [3]:
# Split the train dataset into train and validation
train_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42, shuffle=True)

# Print the characteristics of the dataset
print("Train dataset:", train_dataset)

Train dataset: DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 1115
    })
})


In [4]:
# Print an example of the dataset
print("Example of the dataset:", train_dataset["train"][0])

Example of the dataset: {'sms': 'K..then come wenever u lik to come and also tel vikky to come by getting free time..:-)\n', 'label': 0}


In [5]:
# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the train and test datasets
def tokenize_function(examples):
     return tokenizer(examples["sms"], return_tensors="pt", truncation=True, padding=True)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["sms"])

tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])



In [6]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 4459
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1115
    })
})


In [7]:
# create train and test datasets
train_dataset = tokenized_dataset["train"]
eval_test_split = tokenized_dataset["test"].train_test_split(test_size=0.5, seed=42, shuffle=True)
eval_dataset = eval_test_split["train"]
test_dataset = eval_test_split["test"]

print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 557
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 558
})


In [8]:
len(train_dataset.unique("label"))

2

In [9]:
print("Classes:", train_dataset.unique("label"))

Classes: [0, 1]


In [10]:
# Load the GPT2 model

model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "ham", 1: "spam"},
    label2id={"ham": 0, "spam": 1},
)

model.config.pad_token_id = tokenizer.eos_token_id


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Define a function to compute the metrics

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    eval_loss = np.mean((predictions - labels) ** 2)
    return {"eval_loss": eval_loss}


In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="loss"
)

# Define the Trainer


trainer = Trainer(
    model=model,                                    
    args=training_args,                             
    train_dataset=train_dataset,          
    eval_dataset=eval_dataset,
    data_collator=DataCollatorWithPadding(tokenizer, padding=True, return_tensors="pt"),
    compute_metrics=compute_metrics,
)

In [13]:
# Train the model
trainer.train()

Step,Training Loss,Validation Loss
500,0.158500,0.084535
1000,0.083900,0.074981
1500,0.067900,0.058944
2000,0.035600,0.047157
2500,0.012200,0.047660
3000,0.039100,0.056038


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=3345, training_loss=0.061205879348275906, metrics={'train_runtime': 868.9694, 'train_samples_per_second': 15.394, 'train_steps_per_second': 3.849, 'total_flos': 1638153026076672.0, 'train_loss': 0.061205879348275906, 'epoch': 3.0})

In [14]:
# Evaluate the model
result = trainer.evaluate()

# Print the evaluation results
print(result)

{'eval_loss': 0.04715709388256073, 'eval_runtime': 5.904, 'eval_samples_per_second': 94.343, 'eval_steps_per_second': 11.856, 'epoch': 3.0}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [15]:
config = LoraConfig(
    task_type="SEQ_CLS", 
    r=8,
    lora_alpha=32,
    target_modules=None,
    lora_dropout=0.01,
)

model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model.config.pad_token_id = model.config.eos_token_id
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
lora_model = get_peft_model(model, config)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [16]:
# Number of trainable parameters
lora_model.print_trainable_parameters()

trainable params: 297,984 || all params: 124,737,792 || trainable%: 0.23888830740245906


In [17]:
train_lora = tokenized_dataset["train"].rename_column('label', 'labels')
test_lora = tokenized_dataset["test"].rename_column('label', 'labels')

In [18]:
# Load the metric you want to use
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
# Training Arguments
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./lora",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    remove_unused_columns=False,
    label_names=["labels"],
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



In [20]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.411500,0.081217,0.982063
1000,0.063800,0.091652,0.984753
1500,0.045200,0.087126,0.989238


Checkpoint destination directory ./lora/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./lora/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./lora/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=1674, training_loss=0.1651121000520075, metrics={'train_runtime': 553.4889, 'train_samples_per_second': 24.169, 'train_steps_per_second': 3.024, 'total_flos': 1729735358681088.0, 'train_loss': 0.1651121000520075, 'epoch': 3.0})

In [21]:
lora_model.save_pretrained("./models/gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [22]:
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("./models/gpt-lora")
lora_model.config.pad_token_id = lora_model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [34]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./lora_saved_mdl",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    remove_unused_columns=False,
    label_names=["labels"],
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_lora,
    eval_dataset=test_lora,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [35]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
500,0.064000,0.088437,0.982960
1000,0.070200,0.087482,0.985650
1500,0.056900,0.088460,0.985650


TrainOutput(global_step=1674, training_loss=0.06714766626574517, metrics={'train_runtime': 284.6592, 'train_samples_per_second': 46.993, 'train_steps_per_second': 5.881, 'total_flos': 1729735358681088.0, 'train_loss': 0.06714766626574517, 'epoch': 3.0})

In [36]:
trainer.evaluate()

{'eval_loss': 0.08748237043619156,
 'eval_accuracy': 0.9856502242152466,
 'eval_runtime': 11.652,
 'eval_samples_per_second': 95.692,
 'eval_steps_per_second': 12.015,
 'epoch': 3.0}